In [ ]:
import pandas as pd
import rhoa
import matplotlib.pyplot as plt

# Load your price data
df = rhoa.data.import_sheet("https://docs.google.com/spreadsheets/d/1To-jrrwNBP1XbC4g7LOFdruBKAOdfe-zW3dJrBEkjC4/edit?gid=0#gid=0")[2000:]

# Compute RSI
df["rsi"] = df["Close"].indicators.rsi()

# Remove NaN values
df = df.dropna()

print(f"Data after cleaning: {df.shape[0]} rows")

# === Simple RSI Strategy ===
# Buy when RSI < 30 (oversold)
# Sell when RSI > 70 (overbought)
df["signal"] = 0
df.loc[df["rsi"] < 30, "signal"] = 1   # Buy signal
df.loc[df["rsi"] > 70, "signal"] = -1  # Sell signal

# Position (shifted to avoid lookahead bias)
df["position"] = df["signal"].shift().fillna(0).ffill()

# Debug info
buy_signals = (df["signal"] == 1).sum()
sell_signals = (df["signal"] == -1).sum()
position_changes = (df["position"] != df["position"].shift()).sum()

print(f"\nBuy signals (RSI < 30): {buy_signals}")
print(f"Sell signals (RSI > 70): {sell_signals}")
print(f"Position changes: {position_changes}")
print(f"Unique positions: {df['position'].unique()}")

# Returns calculation
df["returns"] = df["Close"].pct_change()
df["strategy_returns"] = df["position"] * df["returns"]
df["cumulative_returns"] = (1 + df["strategy_returns"]).cumprod()

# Buy & Hold comparison
df["buy_hold"] = df["Close"] / df["Close"].iloc[0]